In [1]:
from deep_cfr import *

def check_main(iter, trav, train_v=2000, batch_v=1000, train_s=2000, batch_s=1000):
    tf.set_random_seed(1)
    np.random.seed(1)
    G = game()
    errs0 = []
    errs1 = []
    B_v = (buffer(), buffer())
    B_s = buffer()
    W = [[],[],[]]
    M_r = (model('p0', True), model('p1', True))
    M_s = model('state')
    for t in range(iter):
        p = t%2
        p_not = (p+1)%2
        B_vp = B_v[p]
        B_s.set()
        B_vp.set()
        for n in range(trav):
            collect_samples(G, "A", p, p_not, M_r, B_vp, B_s)
        W[p].extend([(1+t)/2]*B_vp.count)
        W[2].extend([(1+t)/2]*B_s.count)
        M_r[p].train(B_vp, W[p], train_v, batch_v)
        if t % 100 == 0:
                M_s.train(B_s, W[2], train_s, batch_s, True)
                err0, err1 = measure_performance(M_s)
                errs0.append(err0)
                errs1.append(err1)
    print("value for p0", B_v[0])
    print("value for p1", B_v[1])
    print("W for p0", W[0])
    print("W for p1", W[1])
    print(errs0, errs1)
    
check_main(2, 2)

p0: 0000 mse: 0.410
p0: 1000 mse: 0.189
state: 0000 mse: 0.789
state: 1000 mse: 0.000
p1: 0000 mse: 0.674
p1: 1000 mse: 0.251
value for p0 <objects.buffer object at 0x7efcec25e8d0>
value for p1 <objects.buffer object at 0x7efc9af016a0>
W for p0 [0.5, 0.5, 0.5]
W for p1 [1.0, 1.0]


NameError: name 'errs' is not defined

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
